[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_built_in_sign.ipynb)

In [ ]:
%pip install pdftools_sdk
%pip install ipython

# Sign a PDF using a software-based certificate file
Add a document signature, sometimes called an approval
signature.
This type of signature verifies the integrity of the
signed part of the document and authenticates the
signer's identity.

Validation information is embedded to enable the
long-term validation (LTV) of the signature.

The signing certificate is read from a password-protected
PKCS#12 file (.pfx or .p12).

In [ ]:
import io
from pdftools_sdk.pdf import Document
from pdftools_sdk.sign import Signer, Appearance
from pdftools_sdk.crypto.providers.built_in import Provider
from pdftools_sdk.crypto import ValidationInformation
from pdftools_sdk.geometry.units.size import Size

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
certificate_file = 'INSERT-PFX-FILE'  # Placeholder for PFX file
password = 'INSERT-PASSWORD'
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_path)
output_path = 'Signed.pdf'

In [ ]:
def sign(certificate_file: str, password: str, input_path: str, output_path: str):
    # Create a session to the built-in cryptographic provider
    with Provider() as session:
        with io.FileIO(certificate_file, 'rb') as pfx_str:
            # Create signature configuration from PFX (or P12) file
            signature = session.create_signature_from_certificate(pfx_str, password)

            # Embed validation information to enable long-term validation (LTV) of the signature
            signature.validation_information = ValidationInformation.EMBED_IN_DOCUMENT

            signature.appearance = Appearance.create_field_bounding_box(Size(width=200, height=300))
            signature.appearance.page_number = 1

            # Open input document
            with io.FileIO(input_path, 'rb') as input_pdf_stream:
                with Document.open(input_pdf_stream) as input_pdf_document:
                    # Create stream for output file
                    with io.FileIO(output_path, 'wb+') as output_stream:
                        # Sign the input document
                        signer = Signer()
                        signer.sign(input_pdf_document, signature, output_stream)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    # Sign a PDF document
    sign(certificate_file, password, input_path, output_path)

    print(f"Successfully created signed document {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")